In [82]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
from scipy.stats.kde import gaussian_kde
from pandas import Series
import math
import matplotlib.pyplot as plt

In [2]:
#整理資料
origin_list=pd.read_csv("Materials Equity ETF List (62).csv",index_col=22,parse_dates=True)
origin_list2=pd.read_csv("Telecom Equity ETF List (10).csv",index_col=22,parse_dates=True)
need_list=pd.concat([origin_list,origin_list2])

In [3]:
#切出symbol欄&ETF名稱欄
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values

In [81]:
all_ETF

array(['XLB', 'IYM', 'IGE', 'VAW', 'PKB', 'XHB', 'ITB', 'GDX', 'XME',
       'MXI', 'PYZ', 'SLX', 'RTM', 'UYM', 'SMN', 'FXZ', 'MOO', 'CUT',
       'WOOD', 'HAP', 'PSAU', 'PAGG', 'FLM', 'GRES', 'GDXJ', 'CHIM',
       'FTRI', 'FTAG', 'SBM', 'PSCM', 'SIL', 'COPX', 'LIT', 'GNR', 'REMX',
       'GOEX', 'URA', 'NUGT', 'DUST', 'CROP', 'SOIL', 'GUNR', 'PICK',
       'RING', 'SLVP', 'VEGI', 'SILJ', 'JNUG', 'JDST', 'FMAT', 'SGDM',
       'GDXX', 'GDXS', 'HOML', 'SGDJ', 'NAIL', 'NANR', 'VOX', 'ERUS',
       'IYZ', 'FCOM', 'IXP', 'XTL', 'WBIF', 'FONE', 'LTL'], dtype=object)

In [4]:
#combine ETF daily data
#use the combined data to build the weekly and monthly ETF return 
def combine_daily_data(ETF):
    com_data=pd.DataFrame()
    for s in range(len(ETF)):
        read = "./data/"+ ETF[s] +".csv"
        data = pd.read_csv(read,index_col=0,parse_dates=True)["Adj Close"]
        com_data = pd.concat([com_data,data],axis=1)
    com_data.columns = ETF
    return com_data

Daily_ETF = combine_daily_data(all_ETF)

In [5]:
#fill in the no-trade dates
day=Daily_ETF.index[0]
n=0
while day!=Daily_ETF.index[-1]:
    if (Daily_ETF.index[n+1]-Daily_ETF.index[n]).days !=1:
        for i in range(1,(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days):
            Daily_ETF.loc[Daily_ETF.index[n]+dt.timedelta(i)]=Daily_ETF.loc[Daily_ETF.index[n]]
    Daily_ETF=Daily_ETF.sort_index()
    n = n+(Daily_ETF.index[n+1]-Daily_ETF.index[n]).days
    day=Daily_ETF.index[n]

In [35]:
week_index=[]
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].isoweekday()==5:
        week_index.append(Daily_ETF.index[i])
month_index=[]
for i in range(Daily_ETF.shape[0]):
    if Daily_ETF.index[i].day==1:
        month_index.append(Daily_ETF.index[i])

In [74]:
riskless_rate=pd.read_csv("USG3M.csv",index_col=0,parse_dates=True, )
USG3M=riskless_rate["Price"].reset_index("Date")
USG3M.set_index("Date" , inplace=True)
USG3M=USG3M[::-1]
n=0
day=USG3M.index[0]
while day!=USG3M.index[-1]:
    if (USG3M.index[n+1]-USG3M.index[n]).days !=1:
        for i in range(1,(USG3M.index[n+1]-USG3M.index[n]).days):
            USG3M.loc[USG3M.index[n]+dt.timedelta(i)]=USG3M.loc[USG3M.index[n]]
    USG3M=USG3M.sort_index()
    n = n+(USG3M.index[n+1]-USG3M.index[n]).days
    day=USG3M.index[n]

In [75]:
weekly_ETF=Daily_ETF.loc[week_index]
monthly_ETF=Daily_ETF.loc[month_index]
wr=weekly_ETF/weekly_ETF.shift(1)-1
mr=monthly_ETF/monthly_ETF.shift(1)-1
wUSG3M = USG3M.loc[week_index] / 100 / 52
mUSG3M = USG3M.loc[month_index] / 100 / 12

In [76]:
#Portfolio performance evaluation with generalized Sharpe ratios:
#Beyond the mean and variance
#For weekly data
#3 years for a round

# potential bug r_f , negative number in the sqrt

wASKSR = []

for now, symbol in enumerate(symbol_list):
    wASKSR.append([])
    print(symbol)
    List = wr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l):
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            wASKSR[now].append(float('nan'))
            continue
        if(i + 52*3 >= l):
            break
        X = []
        for j in range(i, i+52*3):
            X.append(List[j])
        X = Series(X)
        #computing ASKSR
        K = X.kurt() #Kurt
        S = X.skew() #Skew
        std = X.std() #stdev
        
        tmp = 3*K-4*S**2
        if tmp < 0 :
            tmp *= -1
        alpha = 3*math.sqrt(tmp) / (std**2 * (3*K - 5 * S**2))
        #print("alpha = ", alpha)

        beta =  3 * S / (std*(3*K-5*S**2))
        #print("beta = ", beta)

        right_tail_n = X.mean() - 3*S*std/(3*K-4*S**2)
        #print("tail_n = ", right_tail_n)

        tmp = 3*K-5*S**2
        if tmp < 0:
            tmp *= -1
        delta = 3 * std * math.sqrt(tmp)/(3*K - 4*S**2)
        #print("delta = ", delta)

        r_f = wUSG3M["Price"][i+52*3] # risk-free interest rate per unit of time, 0.023 is just a testing data

        a_star_multiple_lambda =  (beta + alpha*(right_tail_n - r_f) / math.sqrt(delta**2 + (right_tail_n - r_f)**2) )
        #print(a_star_multiple_lambda)
        
        tmp = alpha**2 - beta**2
        if tmp < 0:
            tmp *= -1
        psi = math.sqrt(tmp)
        #print(psi)
        
        tmp = alpha**2-(beta-a_star_multiple_lambda)**2
        if tmp < 0 :
            tmp *= -1
        tmp = 2*(a_star_multiple_lambda*(right_tail_n - r_f) - delta*(psi - math.sqrt(tmp)))
        if tmp < 0:
            tmp *= -1 
        # equation 38
        ASKSR = math.sqrt(tmp)
        #print("TIME:",List.index[i+52*3],"ASKSR:", ASKSR)
        wASKSR[now].append(ASKSR)
print(wASKSR)

XLB
IYM
IGE
VAW
PKB
XHB
ITB
GDX
XME
MXI
PYZ
SLX
RTM
UYM
SMN
FXZ
MOO
CUT
WOOD
HAP
PSAU
PAGG
FLM
GRES
GDXJ
CHIM
FTRI
FTAG
SBM
PSCM
SIL
COPX
LIT
GNR
REMX
GOEX
URA
NUGT
DUST
CROP
SOIL
GUNR
PICK
RING
SLVP
VEGI
SILJ
JNUG
JDST
FMAT
SGDM
GDXX
GDXS
HOML
SGDJ
NAIL
NANR
VOX
ERUS
IYZ
FCOM
IXP
XTL
WBIF
FONE
LTL
[[nan, 2.7925802831684625, 2.6233656214776553, 2.610903970258865, 2.6518584319899503, 2.862100744860715, 2.6709271470121143, 2.6690706775391937, 2.5296980300087197, 2.6454129889706572, 2.7779396389949125, 2.7538807848568836, 2.7267818630167264, 2.57893228316808, 2.421992482758126, 2.378401736087853, 2.439330980449647, 2.5887503725928287, 2.6036825445300904, 2.646106611103642, 2.7320867232082766, 2.6470605286170934, 2.630730235365912, 2.6365547583211106, 2.5161808408935395, 2.543132734012525, 2.4721060967481936, 2.3061814880291354, 2.153794544756633, 2.241591928767204, 2.1773723204291677, 1.8981355228981225, 1.9774097146973841, 2.051605157995518, 2.125849329929169, 2.260084441960646, 2.372486

In [78]:
#Portfolio performance evaluation with generalized Sharpe ratios:
#Beyond the mean and variance
#For monthly data
#3 years for a round 
mASKSR = []

for now, symbol in enumerate(symbol_list):
    mASKSR.append([])
    print(symbol)
    List = mr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l):
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            mASKSR[now].append(float('nan'))
            continue
        if(i + 12*3 >= l):
            break
        X = []
        for j in range(i, i+12*3):
            X.append(List[j])
        X = Series(X)
        #computing ASKSR
        K = X.kurt() #Kurt
        S = X.skew() #Skew
        std = X.std() #stdev
        
        tmp = 3*K-4*S**2
        if tmp < 0 :
            tmp *= -1
        alpha = 3*math.sqrt(tmp) / (std**2 * (3*K - 5 * S**2))
        #print("alpha = ", alpha)

        beta =  3 * S / (std*(3*K-5*S**2))
        #print("beta = ", beta)

        right_tail_n = X.mean() - 3*S*std/(3*K-4*S**2)
        #print("tail_n = ", right_tail_n)

        tmp = 3*K-5*S**2
        if tmp < 0:
            tmp *= -1
        delta = 3 * std * math.sqrt(tmp)/(3*K - 4*S**2)
        #print("delta = ", delta)

        r_f = mUSG3M["Price"][i+12*3] # risk-free interest rate per unit of time, 0.023 is just a testing data

        a_star_multiple_lambda =  (beta + alpha*(right_tail_n - r_f) / math.sqrt(delta**2 + (right_tail_n - r_f)**2) )
        #print(a_star_multiple_lambda)
        
        tmp = alpha**2 - beta**2
        if tmp < 0:
            tmp *= -1
        psi = math.sqrt(tmp)
        #print(psi)
        
        tmp = alpha**2-(beta-a_star_multiple_lambda)**2
        if tmp < 0 :
            tmp *= -1
        tmp = 2*(a_star_multiple_lambda*(right_tail_n - r_f) - delta*(psi - math.sqrt(tmp)))
        if tmp < 0:
            tmp *= -1 
        # equation 38
        ASKSR = math.sqrt(tmp)
        #print("TIME:",List.index[i+52*3],"ASKSR:", ASKSR)
        mASKSR[now].append(ASKSR)
print(mASKSR)


XLB
IYM
IGE
VAW
PKB
XHB
ITB
GDX
XME
MXI
PYZ
SLX
RTM
UYM
SMN
FXZ
MOO
CUT
WOOD
HAP
PSAU
PAGG
FLM
GRES
GDXJ
CHIM
FTRI
FTAG
SBM
PSCM
SIL
COPX
LIT
GNR
REMX
GOEX
URA
NUGT
DUST
CROP
SOIL
GUNR
PICK
RING
SLVP
VEGI
SILJ
JNUG
JDST
FMAT
SGDM
GDXX
GDXS
HOML
SGDJ
NAIL
NANR
VOX
ERUS
IYZ
FCOM
IXP
XTL
WBIF
FONE
LTL
[[nan, 3.1996813065002514, 3.506164588390253, 1.8095643679317306, 1.8960779288508167, 0.4164792698194876, 0.1698219697622408, 0.04306108865364395, 0.4351905124817482, 1.3214992494400737, 0.1776529907748958, 0.2613547866680316, 0.23869675760071718, 0.38496818971250873, 0.589352342759481, 1.059478072398005, 0.9331873417823198, 1.0952229509928115, 2.6016958693325907, 2.4576945972796715, 3.004415655703443, 2.060332920009265, 18.55984264769477, 14.237184330029109], [nan, 11.653641700284748, 14.12044929448064, 21.469418519305204, 285.0384263950351, 3.637207957728311, 4.210959059992762, 4.654212695191254, 1.4788791486226176, 2.6300290113147793, 0.28166322428329116, 0.12256865870612645, 0.5995457583

In [79]:
#2011 JBF omega performance messure ...
#we need Sharpe_omega(L) which equals to omega_x(L) - 1
#F(.) is the cdf of the asset return X defined on the interval (a,b)
#and omega_x(L) = ( integrate from L to b : (1-F(x))dx) / ( integrate from a to L : F(x)dx )
#where L is the return threshold selected by the investor 
#for weekly:
wOmega = []
for now, symbol in enumerate(symbol_list):
    wOmega.append([])
    print(symbol)
    List = wr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l):
        if(i + 52*3 >= l):
            break
            
        L =  wUSG3M["Price"][i+52*3] #risk-free interest rate 
        
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            wOmega[now].append(float('nan'))
            continue
        X = []
        for j in range(i, i+52*3):
            X.append(List[j])
        X.sort()
        a = X[0]
        b = X[len(X)-1]
        # using 0.001 as a minimum interval 
        left_part = 0
        right_part = 0
        for i in range(len(X)):
            if(X[i] < L):
                left_part +=  L - X[i]
            else :
                right_part += X[i] - L
#        print(right_part, left_part)
        omega = right_part / left_part
        wOmega[now].append(omega)
print(wOmega)

XLB
IYM
IGE
VAW
PKB
XHB
ITB
GDX
XME
MXI
PYZ
SLX
RTM
UYM
SMN
FXZ
MOO
CUT
WOOD
HAP
PSAU
PAGG
FLM
GRES
GDXJ
CHIM
FTRI
FTAG
SBM
PSCM
SIL
COPX
LIT
GNR
REMX
GOEX
URA
NUGT
DUST
CROP
SOIL
GUNR
PICK
RING
SLVP
VEGI
SILJ
JNUG
JDST
FMAT
SGDM
GDXX
GDXS
HOML
SGDJ
NAIL
NANR
VOX
ERUS
IYZ
FCOM
IXP
XTL
WBIF
FONE
LTL
[[nan, 1.1354454629030661, 1.1201931964090008, 1.1182994795131302, 1.1211403733153837, 1.1436987694281642, 1.1267689191106824, 1.125383216579659, 1.113175470010702, 1.1277012365272903, 1.135986904655034, 1.1356539400052406, 1.159542509434449, 1.143209608283094, 1.115845569280193, 1.1118971354892684, 1.1173174667016228, 1.129455735877985, 1.1395168467943493, 1.1448996554216877, 1.1520888162116811, 1.194271688649307, 1.2588777111776142, 1.2588724077274878, 1.232994189620056, 1.2342880787944905, 1.222849304265575, 1.196621086078269, 1.1749601074641842, 1.2151113799925815, 1.206875614563511, 1.280694937559033, 1.2312546544128349, 1.2418417213089574, 1.2586724921959582, 1.301078111593413, 1.32433

In [80]:
#2011 JBF omega performance messure ...
#we need Sharpe_omega(L) which equals to omega_x(L) - 1
#F(.) is the cdf of the asset return X defined on the interval (a,b)
#and omega_x(L) = ( integrate from L to b : (1-F(x))dx) / ( integrate from a to L : F(x)dx )
#where L is the return threshold selected by the investor 
#for monthly:
mOmega = []
for now, symbol in enumerate(symbol_list):
    mOmega.append([])
    print(symbol)
    List = mr[symbol]
    #print(List.shape[0]) #length of the series
    l = List.shape[0]
    for i in range(l):
        if(i + 12*3 >= l):
            break
            
        L =  mUSG3M["Price"][i+12*3] #risk-free interest rate 
        
        if(not ( float("-inf") <List[i] < float("inf") )): #nan
            wOmega[now].append(float('nan'))
            continue
        X = []
        for j in range(i, i+12*3):
            X.append(List[j])
        X.sort()
        a = X[0]
        b = X[len(X)-1] 
        left_part = 0
        right_part = 0
        for i in range(len(X)):
            if(X[i] < L):
                left_part +=  L - X[i]
            else :
                right_part += X[i] - L
        omega = right_part / left_part
        mOmega[now].append(omega)
        
print(mOmega)

XLB
IYM
IGE
VAW
PKB
XHB
ITB
GDX
XME
MXI
PYZ
SLX
RTM
UYM
SMN
FXZ
MOO
CUT
WOOD
HAP
PSAU
PAGG
FLM
GRES
GDXJ
CHIM
FTRI
FTAG
SBM
PSCM
SIL
COPX
LIT
GNR
REMX
GOEX
URA
NUGT
DUST
CROP
SOIL
GUNR
PICK
RING
SLVP
VEGI
SILJ
JNUG
JDST
FMAT
SGDM
GDXX
GDXS
HOML
SGDJ
NAIL
NANR
VOX
ERUS
IYZ
FCOM
IXP
XTL
WBIF
FONE
LTL
[[1.2776256182020365, 1.2530931378641612, 1.335309030759705, 1.2893100291485153, 1.4330777159376975, 1.5125378959836735, 1.4824310087207087, 1.5229397517030672, 1.6193964320475402, 1.3269988860586341, 1.354971596465867, 1.2649062194578173, 1.3623897664121456, 1.414275382886383, 1.6193856455282318, 1.986350803352895, 2.1212346301982916, 1.453852271303513, 1.4280813956460017, 1.3735048205698182, 1.9594301977231825, 1.7489465854230253, 1.6574216692814427], [1.160606500131549, 1.1294540926518637, 1.2222773445544926, 1.1853764216005724, 1.3375562733892776, 1.4150345952694736, 1.4032268655536317, 1.4621382047737455, 1.5850795985796742, 1.3241401099002594, 1.3379813326898986, 1.2670270607685314, 1.

In [ ]:
wtime = []
for i in range(len(weekly_ETF.index)-52*3) :
    wtime.append(weekly_ETF.index[i+52*3])
    #print(time[i])
mtime = []
for i in range(len(monthly_ETF.index)-12*3) :
    mtime.append(monthly_ETF.index[i+12*3])
print(mtime)

In [ ]:
#print(len(symbol_list))
#print(len(mOmega))

mOmega_dic = {}
mOmega_dic.update({'Date' : mtime})
for i in range(len(symbol_list)):
    mOmega_dic.update({symbol_list[i] : mOmega[i]})
mOmega_df = pd.DataFrame.from_dict(mOmega_dic)
mOmega_df.set_index('Date', inplace=True)
#print(mOmega_df)

wOmega_dic = {}
wOmega_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wOmega_dic.update({symbol_list[i] : wOmega[i]})
wOmega_df = pd.DataFrame.from_dict(wOmega_dic)
wOmega_df.set_index('Date', inplace=True)
#print(wOmega_df)

mASKSR_dic = {}
mASKSR_dic.update({ "Date" : mtime})
for i in range(len(symbol_list)):
    mASKSR_dic.update({symbol_list[i] : mASKSR[i]})
mASKSR_df = pd.DataFrame.from_dict(mASKSR_dic)
mASKSR_df.set_index('Date', inplace=True)
#print(mASKSR_df)

wASKSR_dic = {}

wASKSR_dic.update({ "Date" : wtime})
for i in range(len(symbol_list)):
    wASKSR_dic.update({symbol_list[i] : wASKSR[i]})
print()
wASKSR_df = pd.DataFrame.from_dict(wASKSR_dic)
wASKSR_df.set_index('Date', inplace=True)
print(wASKSR_df)

In [ ]:
# ranking the weekly omega
Ranking_wOmega = []
reverse_ranking = []
for i in range(len(symbol_list)):
    Ranking_wOmega.append([])
    reverse_ranking.append(len(symbol_list))
#print(Ranking_wOmega)
#print(len(Ranking_wOmega))

for i in range(len(wtime)):
    #print(i)
    tmp = []
    nan_num = 0
    for j in range(len(symbol_list)):
        tmp.append(wOmega[j][i])
    #print(tmp)
    rank = ss.rankdata(tmp)
    for j in range(len(symbol_list)):
        Ranking_wOmega[i].append(int ( len(symbol_list) - rank[j] + 1) )
    print("Ranking:",Ranking_wOmega[i])
    count = 0
    for j in range(len(symbol_list)):
        if(not ( float("-inf") < wOmega[j][i] < float("inf") )) : #nan
            Ranking_wOmega[i][j] = float("nan")
            count += 1
    # if nan exists, reranking the data
    for j in range(len(symbol_list)):
        if( float("-inf") < wOmega[j][i] < float("inf") ) : #nan
            Ranking_wOmega[i][j] -= count
    #print(Ranking_wOmega[i])

Trans_Ranking = []
for i in range(len(symbol_list)):
    Trans_Ranking.append([])
    for j in len(wtime):
        Trans_Ranking[i].append(Ranking_wOmega[i][j])
    print(Trans_Ranking[i])